In [2]:
from pyvis.network import Network
import networkx as nx
from community import community_louvain
import json

# Création du graphe NetworkX
G = nx.Graph()

# Définition des nœuds et leurs catégories
javascript_nodes = ['javascript', 'cufflinks', 'plotly', 'bokeh', 'bqplot', 'ipyvolume', 
                   'Lighting', 'ipyleaflet', 'pythreejs', 'toyplot']
opengl_nodes = ['OpenGL', 'pyglet', 'vispy', 'glumpy', 'visvis', 'GR Framework', 'mayavi', 'galry']
d3_nodes = ['d3js', 'd3po', 'vincent', 'vega', 'vega-lite', 'altair', 'mpld3']
matplotlib_nodes = ['Matplotlib', 'YT', 'cartopy', 'basemap', 'networkx', 'yellowbrick',
                   'scikit-plot', 'plotnine', 'seaborn', 'pandas', 'glueviz', 'ggpy']
other_nodes = ['holoviews', 'datashader', 'Vaex', 'graph-tool', 'graphviz']
deprecated_nodes = ['pygal', 'chaco', 'PyQTGraph']

# Ajout des nœuds avec leurs catégories
all_nodes = {
    'javascript': javascript_nodes,
    'opengl': opengl_nodes,
    'd3': d3_nodes,
    'matplotlib': matplotlib_nodes,
    'other': other_nodes,
    'deprecated': deprecated_nodes
}

for category, nodes in all_nodes.items():
    for node in nodes:
        G.add_node(node, category=category)

# Définition des liens avec les nouvelles connexions
edges = [
    ('javascript', 'cufflinks'), ('javascript', 'plotly'), ('javascript', 'bokeh'),
    ('javascript', 'bqplot'), ('javascript', 'ipyvolume'), ('javascript', 'Lighting'),
    ('javascript', 'ipyleaflet'), ('javascript', 'pythreejs'), ('javascript', 'toyplot'),
    ('OpenGL', 'pyglet'), ('OpenGL', 'vispy'), ('OpenGL', 'glumpy'), ('OpenGL', 'visvis'),
    ('OpenGL', 'GR Framework'), ('OpenGL', 'mayavi'), ('OpenGL', 'galry'),
    ('d3js', 'd3po'), ('d3js', 'vincent'), ('d3js', 'vega'),
    ('vega', 'vega-lite'), ('vega-lite', 'altair'),
    ('Matplotlib', 'mpld3'), ('Matplotlib', 'YT'), ('Matplotlib', 'cartopy'),
    ('Matplotlib', 'basemap'), ('Matplotlib', 'networkx'), ('Matplotlib', 'yellowbrick'),
    ('Matplotlib', 'scikit-plot'), ('Matplotlib', 'plotnine'), ('Matplotlib', 'seaborn'),
    ('Matplotlib', 'pandas'), ('Matplotlib', 'glueviz'), ('Matplotlib', 'ggpy'),
    ('bokeh', 'holoviews'), ('bokeh', 'datashader'), ('bokeh', 'Vaex'),
    ('holoviews', 'Matplotlib'),
    ('networkx', 'graphviz'),
    ('graphviz', 'graph-tool'),
    ('bqplot', 'Vaex'),
    ('Vaex', 'Matplotlib'),
    ('holoviews', 'datashader'),
    ('bokeh', 'Vaex'),
    ('javascript', 'd3js'),
    ('mpld3', 'd3js')
]

G.add_edges_from(edges)

# Détection des communautés
communities = community_louvain.best_partition(G)

# Création du réseau PyVis avec des dimensions adaptées au notebook
net = Network(
    height='800px',
    width='100%',
    bgcolor='#ffffff',
    font_color='#000000',
    notebook=True
)

# Configuration plus souple de la physique
net.force_atlas_2based(
    gravity=-2000,
    central_gravity=0.1,
    spring_length=200,
    spring_strength=0.05,
    damping=0.95,
    overlap=1.0
)

# Ajout des nœuds sans position fixe
for node in G.nodes():
    base_size = 20 if G.degree(node) > 2 else 15
    size = base_size + 25 * G.degree(node) / max(dict(G.degree()).values())
    
    community_id = communities[node]
    color_intensity = int(180 + (75 * community_id / max(communities.values())))
    color = f'rgb({color_intensity}, {color_intensity}, {color_intensity})'
    
    net.add_node(
        node,
        label=node,
        size=size,
        color=color,
        title=f"Group: {communities[node]}\nDegree: {G.degree(node)}",
        font={'size': 72},
        mass=3 if G.degree(node) > 2 else 1
    )

# Ajout des liens
for edge in G.edges():
    is_main_link = any(node in ['javascript', 'Matplotlib', 'OpenGL', 'd3js'] for node in edge)
    width = 2 if is_main_link else 0.5
    net.add_edge(edge[0], edge[1], width=width)

# Configuration des options
physics_options = {
    "physics": {
        "forceAtlas2Based": {
            "gravitationalConstant": -2000,
            "centralGravity": 0.1,
            "springLength": 200,
            "springConstant": 0.05,
            "damping": 0.95,
            "avoidOverlap": 1.0
        },
        "maxVelocity": 150,
        "minVelocity": 0.75,
        "solver": "forceAtlas2Based",
        "stabilization": {
            "enabled": True,
            "iterations": 1000,
            "updateInterval": 25,
            "fit": True
        },
        "timestep": 0.5,
        "adaptiveTimestep": True
    },
    "nodes": {
        "font": {
            "size": 72,
            "face": "arial"
        },
        "scaling": {
            "min": 15,
            "max": 45
        }
    },
    "edges": {
        "color": {
            "color": "#666666",
            "highlight": "#000000"
        },
        "smooth": {
            "enabled": True,
            "type": "continuous",
            "roundness": 0.5
        }
    },
    "interaction": {
        "dragNodes": True,
        "dragView": True,
        "zoomView": True,
        "navigationButtons": False
    },
    "manipulation": {
        "enabled": False
    }
}

# Conversion des options en JSON
net.set_options(json.dumps(physics_options))

# Style CSS
html_content = """
<style>
.graph-container {
    font-family: Arial, sans-serif;
    padding: 20px;
    background-color: white;
}
.title {
    text-align: center;
    font-size: 36px;
    margin-bottom: 10px;
    color: #333;
}
.subtitle {
    text-align: center;
    font-size: 24px;
    color: #666;
    margin-bottom: 30px;
    font-weight: normal;
}
div.vis-navigation {
    display: none !important;
}
div.vis-network {
    background-color: white !important;
}
#mynetwork {
    background-color: white !important;
}
</style>
<div class="graph-container">
    <div class="title">Visualization Libraries Ecosystem</div>
    <div class="subtitle">Relationships and Dependencies between Major Python and JavaScript Visualization Libraries</div>
</div>
"""

# Sauvegarde initiale
html_file = "temp_network.html"
net.save_graph(html_file)

# Lecture et modification du fichier
with open(html_file, 'r', encoding='utf-8') as f:
    content = f.read()

# Insertion des styles et du titre
content = content.replace('<head>', f'<head>{html_content}')

# Script de stabilisation
js_code = """
<script>
window.addEventListener('load', function() {
    setTimeout(function() {
        network.setOptions({
            physics: {
                stabilization: {
                    enabled: true,
                    iterations: 100,
                    updateInterval: 10
                }
            }
        });
        network.stabilize(100);
        setTimeout(function() {
            network.setOptions({ physics: { enabled: false } });
        }, 100);
    }, 3000);
});
</script>
"""
content = content.replace('</body>', f'{js_code}</body>')

# Sauvegarde finale
with open(html_file, 'w', encoding='utf-8') as f:
    f.write(content)